In [196]:
from deap import base, creator, tools, algorithms
import random
import numpy as np
import pandas as pd

In [197]:
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register('Promedio', np.mean)
stats.register('Desviación Estándar', np.std)
stats.register('Mínimo', np.min)
stats.register('Máximo', np.max)

In [198]:

n_areas = 4
presupuesto = 8000
calidad_minima = 20

# 0=Empleados, 1=Tecnología, 2=Subcontratacion
data = pd.DataFrame([
    {'area': 'A1', 'opcion': 0, 'costo': 1000, 'calidad': 5},
    {'area': 'A1', 'opcion': 1, 'costo': 1500, 'calidad': 7},
    {'area': 'A1', 'opcion': 2, 'costo': 1800, 'calidad': 9},
    {'area': 'A2', 'opcion': 0, 'costo': 800,  'calidad': 4},
    {'area': 'A2', 'opcion': 1, 'costo': 1200, 'calidad': 6},
    {'area': 'A2', 'opcion': 2, 'costo': 1600, 'calidad': 8},
    {'area': 'A3', 'opcion': 0, 'costo': 900,  'calidad': 3},
    {'area': 'A3', 'opcion': 1, 'costo': 1300, 'calidad': 6},
    {'area': 'A3', 'opcion': 2, 'costo': 2000, 'calidad': 10},
    {'area': 'A4', 'opcion': 0, 'costo': 700,  'calidad': 2},
    {'area': 'A4', 'opcion': 1, 'costo': 1100, 'calidad': 5},
    {'area': 'A4', 'opcion': 2, 'costo': 1700, 'calidad': 7}
])

resources = []
for area in data['area'].unique():
    recursos_area = data[data['area'] == area][['costo', 'calidad']].values.tolist()
    resources.append(recursos_area)



In [199]:
def evaluar_individuo(individuo, recursos, presupuesto, calidad_minima):
    costo_total = 0
    calidad_total = 0

    for i, opcion in enumerate(individuo):
        costo, calidad = recursos[i][opcion]
        costo_total += costo
        calidad_total += calidad   

    # Penalizar si no cumple restricciones
    if costo_total > presupuesto or calidad_total < calidad_minima:
        return 0, 999999  

    
    return calidad_total, costo_total,



In [200]:


# Crear tipos
creator.create("FitnessMulti", base.Fitness, weights=(1.0, -1.0))  
creator.create("Individual", list, fitness=creator.FitnessMulti)

# Toolbox
toolbox = base.Toolbox()
toolbox.register("attribute", random.randint, 0, 2)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attribute, n=4)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Evaluador
toolbox.register("evaluate", evaluar_individuo, recursos=resources,
                 presupuesto=presupuesto, calidad_minima=calidad_minima)

toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=2, indpb=0.2)

c:\Users\camil\Documents\EIA\7to\Analisis Decisiones\venv\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\camil\Documents\EIA\7to\Analisis Decisiones\venv\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [201]:
population = toolbox.population(n=8)
population, logbook = algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2, ngen=200, stats = stats, verbose=True)


gen	nevals	Promedio	Desviación Estándar	Mínimo	Máximo
0  	8     	2737.62 	2777.72            	20    	7100  
1  	6     	3303    	3297.64            	26    	7100  
2  	4     	3391    	3386.24            	26    	7100  
3  	7     	3321.94 	3328.6             	23    	7100  
4  	5     	3390.94 	3380.21            	26    	7100  
5  	8     	3523    	3493.34            	30    	7100  
6  	6     	3567    	3533               	34    	7100  
7  	6     	3472.75 	3447.45            	30    	7100  
8  	5     	3567    	3533               	34    	7100  
9  	8     	3460.19 	3450.29            	25    	7100  
10 	7     	3567    	3533               	34    	7100  
11 	8     	3497.81 	3474.22            	27    	7100  
12 	8     	3548.12 	3515.08            	32    	7100  
13 	5     	3397.31 	3393.11            	23    	7100  
14 	5     	3541.88 	3509.37            	32    	7100  
15 	8     	3378.62 	3365.14            	28    	7100  
16 	8     	3567    	3533               	34    	7100  
17 	5     	3567    	3533    

In [202]:
# Filtrar individuos válidos
validos = [ind for ind in population if evaluar_individuo(ind, resources, presupuesto, calidad_minima)[1] < 999999]

print(f"Número de soluciones válidas: {len(validos)}")

if validos:
    # Ahora sí: el mejor entre los válidos
    mejor_valido = tools.selBest(validos, 1)[0]
    print(f"Mejor combinación válida: {mejor_valido}")
    print(f"Calidad y costo: {evaluar_individuo(mejor_valido, resources, presupuesto, calidad_minima)}")
    
    # Mostrar recursos seleccionados
    for i, opcion in enumerate(mejor_valido):
        costo, calidad = resources[i][opcion]
        print(f"Área {i+1}: Opción {opcion} - Costo: {costo}, Calidad: {calidad}")
    
else:
    print("No se encontraron soluciones válidas.")



Número de soluciones válidas: 8
Mejor combinación válida: [2, 2, 2, 2]
Calidad y costo: (34, 7100)
Área 1: Opción 2 - Costo: 1800, Calidad: 9
Área 2: Opción 2 - Costo: 1600, Calidad: 8
Área 3: Opción 2 - Costo: 2000, Calidad: 10
Área 4: Opción 2 - Costo: 1700, Calidad: 7
